In [15]:
# Initial imports.
import pandas as pd
import numpy as np
import datetime as dt
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [16]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("./AAPL_Mock_ML_High.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head()

Date  Season Quarter      Open      High       Low     Close  \
0 1981-02-11  Winter      Q1  0.118304  0.118304  0.117746  0.117746   
1 1981-02-12  Winter      Q1  0.117188  0.117188  0.116629  0.116629   
2 1981-02-13  Winter      Q1  0.114955  0.114955  0.113839  0.113839   
3 1981-02-17  Winter      Q1  0.116629  0.117188  0.116629  0.116629   
4 1981-02-18  Winter      Q1  0.121652  0.122768  0.121652  0.121652   

   Adj Close    Volume  High_24hr_Change Gain_Loss_High  
0   0.092428  13843200          0.118304           Gain  
1   0.091552  14560000         -0.001116           Loss  
2   0.089362  11155200         -0.002233           Loss  
3   0.091552  12275200          0.002233           Gain  
4   0.095495  19241600          0.005580           Gain

In [17]:
apple_df['Date'] = pd.to_datetime(apple_df['Date'])
apple_df['Date'] = apple_df['Date'].map(dt.datetime.toordinal)
apple_df.head()

Date  Season Quarter      Open      High       Low     Close  Adj Close  \
0  723222  Winter      Q1  0.118304  0.118304  0.117746  0.117746   0.092428   
1  723223  Winter      Q1  0.117188  0.117188  0.116629  0.116629   0.091552   
2  723224  Winter      Q1  0.114955  0.114955  0.113839  0.113839   0.089362   
3  723228  Winter      Q1  0.116629  0.117188  0.116629  0.116629   0.091552   
4  723229  Winter      Q1  0.121652  0.122768  0.121652  0.121652   0.095495   

     Volume  High_24hr_Change Gain_Loss_High  
0  13843200          0.118304           Gain  
1  14560000         -0.001116           Loss  
2  11155200         -0.002233           Loss  
3  12275200          0.002233           Gain  
4  19241600          0.005580           Gain

In [18]:
# Must remain in order to activate label_binarize
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_High'] = le.fit_transform(apple_gain_loss_df['Gain_Loss_High'])
apple_gain_loss_df.head()

Date  Season Quarter      Open      High       Low     Close  Adj Close  \
0  723222  Winter      Q1  0.118304  0.118304  0.117746  0.117746   0.092428   
1  723223  Winter      Q1  0.117188  0.117188  0.116629  0.116629   0.091552   
2  723224  Winter      Q1  0.114955  0.114955  0.113839  0.113839   0.089362   
3  723228  Winter      Q1  0.116629  0.117188  0.116629  0.116629   0.091552   
4  723229  Winter      Q1  0.121652  0.122768  0.121652  0.121652   0.095495   

     Volume  High_24hr_Change  Gain_Loss_High  
0  13843200          0.118304               0  
1  14560000         -0.001116               1  
2  11155200         -0.002233               1  
3  12275200          0.002233               0  
4  19241600          0.005580               0

In [19]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_gain_loss_df['Gain_Loss_High']), classes=[1,0])
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_High'] = binarized_gain_loss
apple_gain_loss_df.head()

Date  Season Quarter      Open      High       Low     Close  Adj Close  \
0  723222  Winter      Q1  0.118304  0.118304  0.117746  0.117746   0.092428   
1  723223  Winter      Q1  0.117188  0.117188  0.116629  0.116629   0.091552   
2  723224  Winter      Q1  0.114955  0.114955  0.113839  0.113839   0.089362   
3  723228  Winter      Q1  0.116629  0.117188  0.116629  0.116629   0.091552   
4  723229  Winter      Q1  0.121652  0.122768  0.121652  0.121652   0.095495   

     Volume  High_24hr_Change  Gain_Loss_High  
0  13843200          0.118304               1  
1  14560000         -0.001116               0  
2  11155200         -0.002233               0  
3  12275200          0.002233               1  
4  19241600          0.005580               1

In [20]:
# Perform binary encoding of Season and Quarter columns
apple_binary_encoded = pd.get_dummies(apple_gain_loss_df, columns=["Season", "Quarter"])
apple_binary_encoded.head()

Date      Open      High       Low     Close  Adj Close    Volume  \
0  723222  0.118304  0.118304  0.117746  0.117746   0.092428  13843200   
1  723223  0.117188  0.117188  0.116629  0.116629   0.091552  14560000   
2  723224  0.114955  0.114955  0.113839  0.113839   0.089362  11155200   
3  723228  0.116629  0.117188  0.116629  0.116629   0.091552  12275200   
4  723229  0.121652  0.122768  0.121652  0.121652   0.095495  19241600   

   High_24hr_Change  Gain_Loss_High  Season_Spring  Season_Winter  Quarter_Q1  \
0          0.118304               1              0              1           1   
1         -0.001116               0              0              1           1   
2         -0.002233               0              0              1           1   
3          0.002233               1              0              1           1   
4          0.005580               1              0              1           1   

   Quarter_Q2  
0           0  
1           0  
2           0  
3           0  
4           0

In [21]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_gain_loss_df['Gain_Loss_High']), classes=[1,0])
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_High'] = binarized_gain_loss
apple_gain_loss_df.head()

Date  Season Quarter      Open      High       Low     Close  Adj Close  \
0  723222  Winter      Q1  0.118304  0.118304  0.117746  0.117746   0.092428   
1  723223  Winter      Q1  0.117188  0.117188  0.116629  0.116629   0.091552   
2  723224  Winter      Q1  0.114955  0.114955  0.113839  0.113839   0.089362   
3  723228  Winter      Q1  0.116629  0.117188  0.116629  0.116629   0.091552   
4  723229  Winter      Q1  0.121652  0.122768  0.121652  0.121652   0.095495   

     Volume  High_24hr_Change  Gain_Loss_High  
0  13843200          0.118304               0  
1  14560000         -0.001116               1  
2  11155200         -0.002233               1  
3  12275200          0.002233               0  
4  19241600          0.005580               0

In [22]:
# Define features set
X = apple_binary_encoded.copy()

In [23]:
y = apple_gain_loss_df["Gain_Loss_High"]
X = X.drop(["Gain_Loss_High","High_24hr_Change"], axis=1)
X.head()

Date      Open      High       Low     Close  Adj Close    Volume  \
0  723222  0.118304  0.118304  0.117746  0.117746   0.092428  13843200   
1  723223  0.117188  0.117188  0.116629  0.116629   0.091552  14560000   
2  723224  0.114955  0.114955  0.113839  0.113839   0.089362  11155200   
3  723228  0.116629  0.117188  0.116629  0.116629   0.091552  12275200   
4  723229  0.121652  0.122768  0.121652  0.121652   0.095495  19241600   

   Season_Spring  Season_Winter  Quarter_Q1  Quarter_Q2  
0              0              1           1           0  
1              0              1           1           0  
2              0              1           1           0  
3              0              1           1           0  
4              0              1           1           0

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(30, 11)

In [25]:
#Create a Logistic Regression Model

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [26]:
#Fit (train) or model using the training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [27]:
# Make ppredictions
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

Prediction  Actual
0            1       0
1            1       0
2            1       1
3            1       0
4            1       1
5            1       0
6            1       1
7            1       1
8            1       1
9            1       0
10           1       1

In [28]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.5454545454545454
